In [238]:
import pandas as pd

In [239]:
folder = '.\\input_files\\'
file = '38-4105653_berger_standardcharges.xlsx'

In [240]:
df = pd.read_excel(folder + file, dtype=str, skiprows=3)

In [241]:
df.rename(columns={
    'Procedure Description': 'description',
    'Patient Class (Inpatient/Outpatient)': 'setting',
    'Code': 'code',
    'Quantity': 'drug_quantity',
    'Rev Code': 'rev_code',
    'Derived Negotiated Rate Estimate Min': 'Commercial - Derived Negotiated Rate Estimate Min',
    'Derived Negotiated Rate Estimate Max': 'Commercial - Derived Negotiated Rate Estimate Max',
    'Derived Negotiated Rate Estimate Min.1': 'Managed Medicaid - Derived Negotiated Rate Estimate Min',
    'Derived Negotiated Rate Estimate Max.1': 'Managed Medicaid - Derived Negotiated Rate Estimate Max',
    'Derived Negotiated Rate Estimate Min.2': 'Managed Medicare - Derived Negotiated Rate Estimate Min',
    'Derived Negotiated Rate Estimate Max.2': 'Managed Medicare - Derived Negotiated Rate Estimate Max'
}, inplace=True)

In [242]:
cols = df.columns.tolist()

id_vars = cols[:5]
val_vars = cols[5:]

df = pd.melt(df, id_vars=id_vars, value_vars=val_vars, var_name='payer_name', value_name='standard_charge')

In [243]:
val_vars

['Gross Charge',
 'Discounted Cash Price (Uninsured Discount 35%)',
 'Commercial - Derived Negotiated Rate Estimate Min',
 'Commercial - Derived Negotiated Rate Estimate Max',
 'Managed Medicaid - Derived Negotiated Rate Estimate Min',
 'Managed Medicaid - Derived Negotiated Rate Estimate Max',
 'Managed Medicare - Derived Negotiated Rate Estimate Min',
 'Managed Medicare - Derived Negotiated Rate Estimate Max']

In [244]:
df.dropna(subset=['standard_charge'], inplace=True)

In [245]:
df['setting'] = df['setting'].str.lower()
df['code'] = df['code'].str.strip()

In [246]:
df.loc[(~df['code'].isna()) & (df['code'].str.contains('MS-DRG')), 'ms_drg'] = df['code'].str.extract(r'MS-DRG V39 \(FY2022\) (\d+)', expand=False)
df.loc[(~df['code'].isna()) & (df['code'].str.match('HCPCS|CPT®')), 'hcpcs_cpt'] = df['code'].str.replace('HCPCS |CPT® ', '', regex=True)
df.loc[(~df['code'].isna()) & (df['code'].str.contains('Custom')), 'local_code'] = df['code'].str.replace('Custom ', '')

In [247]:
df['hcpcs_cpt'] = df['hcpcs_cpt'].str.strip()
df.loc[(df['hcpcs_cpt'].str.len() == 8) | (df['hcpcs_cpt'] == 'CUSTOM') | (df['hcpcs_cpt'].str.len() != 5), 'hcpcs_cpt'] = pd.NA

In [248]:
df.loc[(~df['rev_code'].isna()), 'rev_code'] = df['rev_code'].str.extract(r'(\d{4})', expand=False)

In [249]:
df.loc[(df['drug_quantity'] == "1") | (df['drug_quantity'] == '(blank)'), 'drug_quantity'] = pd.NA

In [250]:
df.drop_duplicates(subset='setting')

,description,setting,code,drug_quantity,rev_code,payer_name,standard_charge,ms_drg,hcpcs_cpt,local_code
0,Heart Transplant Or Implant Of Heart Assist Sy...,inpatient,MS-DRG V39 (FY2022) 001,<NA>,NaN,Gross Charge,873485.45,001,<NA>,NaN
738,ETOPOSIDE 20 MG/ML INTRAVENOUS SOLUTION,pharmacy,HCPCS J9181,<NA>,0636,Gross Charge,234.8,NaN,J9181,NaN
1545,BIT 2.5MM DRILL 110MM GOLD QUICK COUPLING,supplies,Custom 27200232,<NA>,0272,Gross Charge,165,NaN,<NA>,27200232
4300,HC ROOM AND BED PRIVATE,outpatient,Custom 11100001,<NA>,0111,Gross Charge,1894,NaN,<NA>,11100001
4338,NaN,NaN,NaN,NaN,0771,Gross Charge,930,NaN,<NA>,NaN
11283,IMM ADMN SARSCOV2 30MCG/0.3ML DIL RECON 1ST DOSE,professional,CPT® 0001A,<NA>,NaN,Gross Charge,80,NaN,0001A,NaN


In [251]:
df.loc[(~df['setting'].isna()) & (df['setting'].str.match(r'pharmacy|supplies')), 'setting'] = pd.NA

In [252]:
df.loc[df['setting'] == 'professional', ['setting', 'billing_class']] = (pd.NA, 'professional')

In [253]:
mapping = {
    'Commercial - Derived Negotiated Rate Estimate Min': 'min',
    'Commercial - Derived Negotiated Rate Estimate Max': 'max',
    'Managed Medicaid - Derived Negotiated Rate Estimate Min': 'min',
    'Managed Medicaid - Derived Negotiated Rate Estimate Max': 'max',
    'Managed Medicare - Derived Negotiated Rate Estimate Min': 'min',
    'Managed Medicare - Derived Negotiated Rate Estimate Max': 'max',
    'Discounted Cash Price (Uninsured Discount 35%)': 'cash',
    'Gross Charge': 'gross',
}

df['payer_category'] = df['payer_name'].map(mapping)

In [260]:
id_mapping = {'38-4105653_berger_standardcharges.xlsx': '360170',
 '31-4394942_doctors_standardcharges.xlsx': '360152',
 '31-4394942_dublin_standardcharges.xlsx': '360348',
 '31-4379436_grady_standardcharges.xlsx': '360210',
 '31-4394942_grant_standardcharges.xlsx': '360017',
 '31-4440479_hardin_standardcharges.xlsx': '361315',
 '31-1070877_marion_standardcharges.xlsx': '360011',
 '31-4394942_grove-city_standardcharges.xlsx': '360372',
 '34-0714456_mansfield_standardcharges.xlsx': '360118',
 '31-4446959_obleness_standardcharges.xlsx': '360014',
 '34-0714456_shelby_standardcharges.xlsx': '361324',
 '31-4394942_riverside_standardcharges.xlsx': '360006'}

hosp_id =  id_mapping[file]
df['id'] = hosp_id

In [262]:
output_folder = ".\\output\\"

file = hosp_id + '_' + file.split('_')[1] + '.csv'

df.to_csv(output_folder + file, index=False)

'360170_berger.csv'